# 3.2.1 缺失值补全
## 单变量插值法（SimpleInputer）
使用参数missing_values指定真实数据中缺失值的标识，使用参数strategy指定插值策略，取值为"mean"、"median"、"most_frequent"或"constant"，分别表示使用每列的平均值、中位数、众数或指定某个常数来替换缺失值，该常数使用fill_value参数指定。

In [1]:
import numpy as np
from sklearn.impute import SimpleImputer
#定义Imputer对象
i=SimpleImputer(missing_values=0,strategy='constant',fill_value=10)# use constant 10 to replace missing value 0
i.fit([[0,1,2]]) # fit()函数让Imputer对象适应(拟合)数据，参数为2维数组
X = np.array([[0,1,2], [6,0,5], [7,8,0]])# 原始数据,列数应与fit函数参数列数相同
X1 = i.transform(X)# 将X中的所有缺失值填补，返回填补后的二维数组 
print(X1)
print(type(X1))

[[10  1  2]
 [ 6 10  5]
 [ 7  8 10]]
<class 'numpy.ndarray'>


fit函数通过设置参数指明了要转换的二维数组的列数，SimpleImputer也提供了fit_transform函数，直接进行拟合数据并对缺省值填补，例如：

In [2]:
i2 = SimpleImputer(missing_values=0, strategy='constant', fill_value=10)
X = np.array([[0,1,2,0], [6,0,5,0], [7,8,0,0]])  # 原始数据,3行4列
X2 = i2.fit_transform(X)# 先fit后进行transform操作，返回填补后的二维数组
print(X2)
print(type(X2))

[[10  1  2 10]
 [ 6 10  5 10]
 [ 7  8 10 10]]
<class 'numpy.ndarray'>


In [3]:
import pandas as pd

i3 = SimpleImputer(missing_values=np.nan,strategy='mean') # mean
print("before",[[np.nan,1,3],[0,np.nan,2],[4,3,1]])
print("after",i3.fit_transform([[np.nan,1,3],[0,np.nan,2],[4,3,1]]))

before [[nan, 1, 3], [0, nan, 2], [4, 3, 1]]
after [[2. 1. 3.]
 [0. 2. 2.]
 [4. 3. 1.]]


In [4]:
# Most Freqent
i4 = SimpleImputer(missing_values='',strategy='most_frequent')
p=pd.DataFrame([['','k','x'],['a','m',''],['b','','y'],['a','m','x']],dtype="category")
print(p)
print("---")
print(i4.fit_transform(p))

   0  1  2
0     k  x
1  a  m   
2  b     y
3  a  m  x
---
[['a' 'k' 'x']
 ['a' 'm' 'x']
 ['b' 'm' 'y']
 ['a' 'm' 'x']]


## 多变量插值（IterativeImputer）

多变量插值IterativeImputer类利用各列非缺省值构建函数为每个缺失值进行建模。建模采用迭代循环方式执行，即在每个步骤中，将目标列指定为输出y，将其他列视为输入X，使用一个回归模型对(Ｘ，ｙ)进行拟合，然后使用这个回归模型来预测缺失的ｙ值。IterativeImputer类也是通过参数missing_values指定真实数据中缺失值的标识，通过参数max_iter控制迭代次数，默认值为10。

以下代码给出了使用多变量插值方法进行数据插补的示例。

In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(missing_values=np.nan)
imp.fit([[1, 3],[2, 5],[3, 7]])
X_test = [[np.nan, 11], [6, np.nan], [np.nan, 6],[3,17]]
print(X_test)
print(imp.transform(X_test))

[[nan, 11], [6, nan], [nan, 6], [3, 17]]
[[ 4.9999985  11.        ]
 [ 6.         12.999999  ]
 [ 2.49999975  6.        ]
 [ 3.         17.        ]]


## 3.2.2 数据无量纲化

许多机器学习算法中的目标函数是假设所有的特征为零均值且具有相同方差。

如果数据某维特征的方差比其他维度的特征大几个数量级，那么这个特征可能会在学习算法中占据**主导**位置，导致模型无法从其他特征中学习到好的分类效果。

因此，我们常常需要对数据进行去量纲的操作，提升模型的收敛速度和精度。

### (1)最小最大归一化（MinMaxScaler）

归一化方法将每列数据缩放到给定的最小值和最大值之间。转换公式为：
                   
$$X_{scaled}=(X-min⁡( X))\frac{Max-Min}{max⁡( X)-min⁡( X)}+Min$$

其中，Min和Max是指定的最小值和最大值。通常使用MinMaxScaler类进行归一化操作，其中一个重要的参数feature_range用于控制缩放的范围，默认值为(0,1)，类型为tuple. 示例如下。

In [6]:
from sklearn.preprocessing import MinMaxScaler
X_train = [[1, -1, 2], [2, 0, 0], [0, 1, -1]]
min_max_scalar = MinMaxScaler(feature_range=(0,1))
X_train_minmax = min_max_scalar.fit_transform(X_train)
print(X_train)
print(X_train_minmax)

[[1, -1, 2], [2, 0, 0], [0, 1, -1]]
[[0.5        0.         1.        ]
 [1.         0.5        0.33333333]
 [0.         1.         0.        ]]


### （2）最大绝对值归一化（MaxAbsScaler）

最大绝对值归一化与最小最大归一化类似，只不过它将每列数据按最大绝对值进行归一化。转换公式表达为：

$$X_{scaled}=\frac{X}{max⁡( abs (X))}$$

因此，每列数据都会映射在[-1,1]范围内

In [7]:
from sklearn.preprocessing import MaxAbsScaler
X_tr = [[1, -1, 2], [2, 0, 0], [0, -2, -1]]
mas = MaxAbsScaler()
X_re = mas.fit_transform(X_tr) # 注意先进行fit，后进行transform
print(X_tr)
print(X_re)

[[1, -1, 2], [2, 0, 0], [0, -2, -1]]
[[ 0.5 -0.5  1. ]
 [ 1.   0.   0. ]
 [ 0.  -1.  -0.5]]


### （3）Z-score标准化（StandardScaler）

Z-score标准化是指将每列数转换为服从为均值为0，方差为1的正态分布（即标准正态分布）的数据。设某列原始数据X的均值为μ，标准差为σ，则转换公式为：

$$X_{std}=\frac{X-μ}{σ}$$  

Z-score标准化是非常常见的归一化方法，示例如下：


In [8]:
from sklearn.preprocessing import StandardScaler as ss
scalar = ss() # 实例化
X_train = [[1, -1, 2], [2, 0, 4], [3, 2, -6]]
scalar.fit(X_train) # 在训练集上训练，得到均值和方差
print('mean', scalar.mean_)  # 查看得到的均值
print('std', scalar.scale_)  # 查看得到的方差

X_train_re = scalar.transform(X_train)# 在训练集上归一化
print('X_train_re', X_re)  # 打印训练集归一化结果
X_test_re = scalar.transform([[1, 2, 3]])  # 在测试集上归一化
print('X_test_re', X_test_re)  # 打印测试集归一化结果

mean [2.         0.33333333 0.        ]
std [0.81649658 1.24721913 4.3204938 ]
X_train_re [[ 0.5 -0.5  1. ]
 [ 1.   0.   0. ]
 [ 0.  -1.  -0.5]]
X_test_re [[-1.22474487  1.33630621  0.69436507]]


## 3.2.3 类别特征编码

### （1）标签编码（LabelEncoder） 

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(["beijing", "shanghai", "beijing", "shenzhen"])
print("labels: ", list(le.classes_))  # 打印所有的标签集合
print(le.transform(["beijing", "shanghai"]))  # 打印给定标签对应的编码
print(list(le.inverse_transform([2, 2, 1])))  # 打印编码对应的原始标签

labels:  ['beijing', 'shanghai', 'shenzhen']
[0 1]
['shenzhen', 'shenzhen', 'shanghai']


### （2）多标签二值化编码（MultiLabelBinarizer）

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

mlb=MultiLabelBinarizer()
Y = [['beijing', 'chengdu'], ['chengdu', 'tianjin'],['chongqing'],['guangzhou'], ['shanghai'], ['shenzhen'], ['tianjin']]

mlb.fit(Y) # 训练标签数据

print('coding:', mlb.transform([['beijing','shanghai']]))  # 打印新标签编码结果


coding: [[1 0 0 0 1 0 0]]


### （3）特征按序编码（OrdinalEncoder） 

In [11]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
X = [['male', 'beijing'], ['female', 'shenzhen'], ['male', 'shanghai']]
oe.fit(X) # 进行训练
print("categories: ", oe.categories_)  # 打印类别
# 打印对新数据的编码结果
print("encode: ", oe.transform([['male','shenzhen'], ['female', 'beijing']]))

categories:  [array(['female', 'male'], dtype=object), array(['beijing', 'shanghai', 'shenzhen'], dtype=object)]
encode:  [[1. 2.]
 [0. 0.]]


### （4）独热编码（OneHotEncoder）
其原理类似于多标签二值化编码，只不过独热编码只能用于数据特征的编码，不能用于标签编码。设数据的某列特征有N个可能的取值，则将该列特征变换为长度为N的二进制特征向量，其中只有一个位置上是1，其余位置都是0，这也是与多标签二值化编码不同的。

In [12]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

X = [['male', 'beijing'], ['female', 'shanghai'], 
['male', 'shenzhen'], ['female', 'beijing']]

ohe.fit(X)

print("categories: ", ohe.categories_)  # 查看特征取值  
print("encode:", ohe.transform([['female','shenzhen']]).toarray()) # 输出特征编码

categories:  [array(['female', 'male'], dtype=object), array(['beijing', 'shanghai', 'shenzhen'], dtype=object)]
encode: [[1. 0. 0. 0. 1.]]


## 3.2.4 离散化

### （1）K-bins离散化（KBinsDiscretizer）
K-bins离散化是指将连续数值型特征值排序后通过使用k个等宽的区间分箱后编码，使得原来连续型变量划分为分类变量。

KBinsDiscretizer类中包含3个重要的参数，其中n_bins参数表示每维特征的分箱个数，默认为5，会被运用到所有导入的特征上。encode参数表示编码方式，可取值为"onehot"、"ordinal"、"onehot-dense"，默认"onehot"。strategy参数表示分箱方式，可取值为"uniform"、"quantile"、"kmeans"，分别表示等宽分箱、等位分箱、按聚类分箱，默认"quantile"。

In [3]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

X = np.array([[-3, 5, 15],[0, 6, 14],[6, 3, 11]])
kbd=KBinsDiscretizer(n_bins=[3,2,2],encode='ordinal')
kbd.fit(X)
print('Edge:', kbd.bin_edges_)  # 显示分箱边界
print('coding:', kbd.transform(X))  # 对训练数据的离散结果
print('coding:', kbd.transform([[-10,2,0],[-1,5.5,16]]))  # 对新数据的离散结果

Edge: [array([-3., -1.,  2.,  6.]) array([3., 5., 6.]) array([11., 14., 15.])]
coding: [[0. 1. 1.]
 [1. 1. 1.]
 [2. 0. 0.]]
coding: [[0. 0. 0.]
 [0. 1. 1.]]


### （2）二值化（Binarizer） 
通过设定阈值将连续型数值特征划分得到布尔值（0或1）的过程。

在计算机视觉中，针对灰度图像的二值化的作用就是将图像变为黑白图像，以便于进行图像分割、目标提取等操作。

Binarizer类提供了参数threshold，用于设置各个维度上的阈值。

In [8]:
from sklearn.preprocessing import Binarizer
X = [[1.,-1.,2.],[2.,0.,0.],[0.,1.,-1.]]
binarizer = Binarizer(threshold=1.0).fit(X)
# 源代码这里好像错了 源代码：threshold=[1,0,1]
print(binarizer.transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]]
